# FARM Building Blocks

Welcome to the FARM building blocks tutorial! There are many different ways to make use of this repository, but in this notebook, we will be going through the most import building blocks that will help you harvest the rewards of a successfully trained NLP model.

Happy FARMing!

## 1) Text Classification

GermEval 2018 (GermEval2018) (https://projects.fzai.h-da.de/iggsa/) is an open data set containing texts that need to be classified by whether they are offensive or not. There are a set of coarse and fine labels, but here we will only be looking at the coarse set which labels each example as either OFFENSE or OTHER. To tackle this task, we are going to build a classifier that is composed of Google's BERT language model and a feed forward neural network prediction head.

### Setup

In [ ]:
# Let's start by adjust the working directory so that it is the root of the repository
# This should be run just once.

import os
os.chdir('../')
print("Current working directory is {}".format(os.getcwd()))

In [ ]:
# Here are the imports we need

import torch
from farm.modeling.tokenization import BertTokenizer
from farm.data_handler.processor import TextClassificationProcessor
from farm.data_handler.data_silo import DataSilo
from farm.modeling.language_model import Bert
from farm.modeling.prediction_head import TextClassificationHead
from farm.modeling.adaptive_model import AdaptiveModel
from farm.modeling.optimization import initialize_optimizer
from farm.train import Trainer

In [17]:
# We need to fetch the right device to drive the growth of our model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
print("Devices available: {}".format(device))

Devices available: cpu


### Data Handling

In [ ]:
# Here we initialize a tokenizer that will be used for preprocessing text
# This is the BERT Tokenizer which uses the byte pair encoding method.
# It is currently loaded with a German model

tokenizer = BertTokenizer.from_pretrained(
    pretrained_model_name_or_path="bert-base-german-cased",
    do_lower_case=False)

In [19]:
# We can test out how it will do on an example sentence

EXAMPLE_SENTENCE = "Selbst ein blindes Huhn findet mal ein Korn."
tokenizer.tokenize(EXAMPLE_SENTENCE)

['Selbst',
 'ein',
 'bl',
 '##inde',
 '##s',
 'Hu',
 '##hn',
 'findet',
 'mal',
 'ein',
 'Korn',
 '.']

In [20]:
# In order to prepare the data for the model, we need a set of
# functions to transform data files into PyTorch Datasets.
# We group these together in Processor objects.
# We will need a new Processor object for each new source of data.
# The abstract class can be found in farm.data_handling.processor.Processor

processor = TextClassificationProcessor(tokenizer=tokenizer,
                                        max_seq_len=128,
                                        data_dir="data/germeval18",
                                        labels = ["OTHER", "OFFENSE"],
                                        metric = "f1_macro",
                                        source_field = "coarse_label")

In [21]:
# We need a DataSilo in order to keep our train, dev and test sets separate.
# The DataSilo will call the functions in the Processor to generate these sets.
# From the DataSilo, we can fetch a PyTorch DataLoader object which will
# be passed on to the model.
# Here is a good place to define a batch size for the model

BATCH_SIZE = 32

data_silo = DataSilo(
    processor=processor,
    batch_size=BATCH_SIZE)

08/31/2019 15:34:03 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
08/31/2019 15:34:03 - INFO - farm.data_handler.data_silo -   Loading train set from: data/germeval18/train.tsv 
08/31/2019 15:34:03 - INFO - farm.data_handler.processor -   Got ya 5 parallel workers to fill the baskets with samples (chunksize = 1000)...
100%|██████████| 5009/5009 [00:01<00:00, 2785.99it/s]
08/31/2019 15:34:05 - INFO - farm.data_handler.processor -   Got ya 5 parallel workers to featurize samples in baskets (chunksize = 1000) ...
  0%|          | 0/5009 [00:00<?, ?it/s]
08/31/2019 15:34:06 - INFO - farm.data_handler.processor -   *** Show 3 random examples ***
08/31/2019 15:34:06 - INFO - farm.data_handler.processor -   

      .--.        _____                       _      
    .'_\/_'.     / ____|                     | |     
    

08/31/2019 15:34:06 - INFO - farm.data_handler.data_silo -   Loading dev set as a slice of train set
08/31/2019 15:34:06 - INFO - farm.data_handler.data_silo -   Took 500 samples out of train set to create dev set (dev split = 0.1)
08/31/2019 15:34:06 - INFO - farm.data_handler.data_silo -   Loading test set from: data/germeval18/test.tsv
08/31/2019 15:34:06 - INFO - farm.data_handler.processor -   Got ya 3 parallel workers to fill the baskets with samples (chunksize = 1000)...
100%|██████████| 3532/3532 [00:01<00:00, 3103.76it/s]
08/31/2019 15:34:08 - INFO - farm.data_handler.processor -   Got ya 3 parallel workers to featurize samples in baskets (chunksize = 1000) ...
  0%|          | 0/3532 [00:00<?, ?it/s]
08/31/2019 15:34:08 - INFO - farm.data_handler.processor -   *** Show 3 random examples ***
08/31/2019 15:34:08 - INFO - farm.data_handler.processor -   

      .--.        _____                       _      
    .'_\/_'.     / ____|                     | |     
    '. /\ .'    |

08/31/2019 15:34:08 - INFO - farm.data_handler.data_silo -   Examples in train: 4509
08/31/2019 15:34:08 - INFO - farm.data_handler.data_silo -   Examples in dev  : 500
08/31/2019 15:34:08 - INFO - farm.data_handler.data_silo -   Examples in test : 3532
08/31/2019 15:34:08 - INFO - farm.data_handler.data_silo -   
08/31/2019 15:34:08 - INFO - farm.data_handler.data_silo -   Max sequence length:     128
08/31/2019 15:34:08 - INFO - farm.data_handler.data_silo -   Average sequence length: 40.17853182523841
08/31/2019 15:34:08 - INFO - farm.data_handler.data_silo -   Proportion clipped:      0.01752051452650255


### Modeling

In FARM, we make a strong distinction between the language model and prediction head so that you can mix and match different building blocks for your needs.

For example, in the transfer learning paradigm, you might have the one language model that you will be using for both document classification and NER. Or you perhaps you have a pretrained language model which you would like to adapt to your domain, then use for a downstream task such as question answering. 

All this is possible within FARM and requires only the replacement of a few modular components, as we shall see below.

Let's first have a look at how we might set up a model.

In [22]:
# The language model is the foundation on which modern NLP systems are built.
# They encapsulate a general understanding of sentence semantics
# and are not specific to any one task.

# Here we are using Google's BERT model as implemented by HuggingFace. 
# The model being loaded is a German model that we trained. 
# You can also change the MODEL_NAME_OR_PATH to point to a BERT model that you
# have saved or download one connected to the HuggingFace repository.
# See farm.modeling.language_model.PRETRAINED_MODEL_ARCHIVE_MAP for a list of
# available models

MODEL_NAME_OR_PATH = "bert-base-german-cased"

language_model = Bert.load(MODEL_NAME_OR_PATH)

08/31/2019 15:34:09 - INFO - pytorch_transformers.modeling_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-german-cased-config.json from cache at /home/mp/.cache/torch/pytorch_transformers/e653e2fe0970d519c5a3b6c0286e1630ad2f0eade78f82b4916ec945d6f06d48.958be95f8721c8cc3ff0998c94a7b77083ecd9345f62423db24ad2387d599c7d
08/31/2019 15:34:09 - INFO - pytorch_transformers.modeling_utils -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "torchscript": false,
  "type_vocab_size": 2,
  "vocab_size": 30000
}

08/31/2019 15:34:09 - INFO - pytorch_transformers.modeling_utils -   loadi

In [23]:
# A prediction head is a model that processes the output of the language model
# for a specific task.
# Prediction heads will look different depending on whether you're doing text classification
# Named Entity Recognition (NER), question answering or some other task.
# They should generate logits over the available prediction classes and contain methods
# to convert these logits to losses or predictions 

# Here we use TextClassificationHead which receives a single fixed length sentence vector
# and processes it using a feed forward neural network. layer_dims is a list of dimensions:
# [input_dims, hidden_1_dims, hidden_2_dims ..., output_dims]

# Here by default we have a single layer network.
# It takes in a vector of length 768 (the default size of BERT's output).
# It outputs a vector of length 2 (the number of classes in the GermEval18 (coarse) dataset)

LAYER_DIMS = [768, 2]

prediction_head = TextClassificationHead(layer_dims=LAYER_DIMS)

In [24]:
# The language model and prediction head are coupled together in the Adaptive Model.
# This class takes care of model saving and loading and also coordinates
# cases where there is more than one prediction head.

# EMBEDS_DROPOUT_PROB is the probability that an element of the output vector from the
# language model will be set to zero.
EMBEDS_DROPOUT_PROB = 0.1

model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_sequence"],
    device=device)

### Training

In [25]:
# Here we initialize a Bert Adam optimizer that has a linear warmup and warmdown
# Here you can set learning rate, the warmup proportion and number of epochs to train for

LEARNING_RATE = 2e-5
WARMUP_PROPORTION = 0.1
N_EPOCHS = 1

optimizer, warmup_linear = initialize_optimizer(
    model=model,
    learning_rate=LEARNING_RATE,
    warmup_proportion=WARMUP_PROPORTION,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=N_EPOCHS)

08/31/2019 15:34:11 - INFO - farm.modeling.optimization -   Number of optimization steps: 141


In [26]:
# Training loop handled by this
# It will also trigger evaluation during training using the dev data
# and after training using the test data.

# Set N_GPU to a positive value if CUDA is available
N_GPU = 0

trainer = Trainer(
    optimizer=optimizer,
    data_silo=data_silo,
    epochs=N_EPOCHS,
    n_gpu=N_GPU,
    warmup_linear=warmup_linear,
    device=device,
)

In [ ]:
model = trainer.train(model)

# Switch to NER

In a transfer learning paradigm, there is a core computation that is shared amongst all tasks. FARM's modular structure means that you can easily swap out different building blocks to make the same language model work for many different tasks.

We can adapt the above text classification model to NER by simply switching out the processor and prediction head.

In [30]:
# Import the new building blocks

from farm.data_handler.processor import NERProcessor
from farm.modeling.prediction_head import TokenClassificationHead

In [31]:
# This processor will preprocess the data for the CoNLL03 NER task

ner_processor = NERProcessor(tokenizer=tokenizer, max_seq_len=128, data_dir="data/conll03-de")

ner_labels = ["[PAD]", "X", "O", "B-MISC", "I-MISC", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC", "B-OTH", "I-OTH"]
processor.add_task("ner", "seq_f1", ner_labels)

In [32]:
# This prediction head is also a feed forward neural network but expects one
# vector per token in the input sequence and will generate a set of logits
# for each input

LAYER_DIMS = [768, 13]

ner_prediction_head = TokenClassificationHead(layer_dims=LAYER_DIMS)

In [33]:
# We can integrate these new pieces with the rest using this code
# It is pretty much the same structure as what we had above for text classification

BATCH_SIZE = 32
EMBEDS_DROPOUT_PROB = 0.1
LEARNING_RATE = 2e-5
WARMUP_PROPORTION = 0.1
N_EPOCHS = 1
N_GPU = 0

data_silo = DataSilo(
    processor=ner_processor,
    batch_size=BATCH_SIZE)

model = AdaptiveModel(
    language_model=language_model,
    prediction_heads=[ner_prediction_head],
    embeds_dropout_prob=EMBEDS_DROPOUT_PROB,
    lm_output_types=["per_token"],
    device=device)

optimizer, warmup_linear = initialize_optimizer(
    model=model,
    learning_rate=LEARNING_RATE,
    warmup_proportion=WARMUP_PROPORTION,
    n_batches=len(data_silo.loaders["train"]),
    n_epochs=N_EPOCHS)

trainer = Trainer(
    optimizer=optimizer,
    data_silo=data_silo,
    epochs=N_EPOCHS,
    n_gpu=N_GPU,
    warmup_linear=warmup_linear,
    device=device,
)

08/31/2019 15:35:55 - INFO - farm.data_handler.data_silo -   
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|       
 (o)(o)------'\ _ /     ( )      
 
08/31/2019 15:35:55 - INFO - farm.data_handler.data_silo -   Loading train set from: data/conll03-de/train.txt 
08/31/2019 15:35:55 - INFO - farm.data_handler.utils -    Couldn't find data/conll03-de/train.txt locally. Trying to download ...
08/31/2019 15:35:55 - INFO - farm.data_handler.utils -   downloading and extracting file conll03-de to dir /home/mp/deepset/dev/FARM/data
08/31/2019 15:36:00 - INFO - farm.data_handler.processor -   Got ya 8 parallel workers to fill the baskets with samples (chunksize = 1000)...

100%|██████████| 24000/24000 [00:06<00:00, 3911.33it/s]08/31/2019 15:36:07 - INFO - farm.data_handler.processor -   Got ya 8 parallel workers to featurize samples in baskets (chunksize = 1000) ...

  0%|          | 0/24000 [00:00<?, ?i

08/31/2019 15:36:10 - INFO - farm.data_handler.processor -   

      .--.        _____                       _      
    .'_\/_'.     / ____|                     | |     
    '. /\ .'    | (___   __ _ _ __ ___  _ __ | | ___ 
      "||"       \___ \ / _` | '_ ` _ \| '_ \| |/ _ \ 
       || /\     ____) | (_| | | | | | | |_) | |  __/
    /\ ||//\)   |_____/ \__,_|_| |_| |_| .__/|_|\___|
   (/\||/                             |_|           
______\||/___________________________________________                     

ID: train-2515-0
Clear Text: 
 	text: Bei Zwischenstopps sei dies außerdem nur dann der Fall gewesen , wenn die Besatzung komplett gewechselt habe .
 	ner_label: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Tokenized: 
 	tokens: ['Bei', 'Zwischen', '##stopp', '##s', 'sei', 'dies', 'außerdem', 'nur', 'dann', 'der', 'Fall', 'gewesen', ',', 'wenn', 'die', 'Besatzung', 'komplett', 'gewechselt', 'habe', '.']
 	offsets: [0, 4, 12, 17, 19, 

08/31/2019 15:36:13 - INFO - farm.data_handler.processor -   

      .--.        _____                       _      
    .'_\/_'.     / ____|                     | |     
    '. /\ .'    | (___   __ _ _ __ ___  _ __ | | ___ 
      "||"       \___ \ / _` | '_ ` _ \| '_ \| |/ _ \ 
       || /\     ____) | (_| | | | | | | |_) | |  __/
    /\ ||//\)   |_____/ \__,_|_| |_| |_| .__/|_|\___|
   (/\||/                             |_|           
______\||/___________________________________________                     

ID: dev-299-0
Clear Text: 
 	text: Geographie Der Ort liegt sechs Kilometer südlich von Modrava im Luzenské údolí ( Lusental ) , das nördlich vom Berg Lusen in Richtung Modrava verläuft .
 	ner_label: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'B-LOC', 'I-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', 'O', 'B-LOC', 'O', 'O']
Tokenized: 
 	tokens: ['Ge', '##ographie', 'Der', 'Ort', 'liegt', 'sechs', 'Kilometer', 'südlich', 'von', 'Mod', '##ra', '##va'

08/31/2019 15:36:17 - INFO - farm.data_handler.processor -   

      .--.        _____                       _      
    .'_\/_'.     / ____|                     | |     
    '. /\ .'    | (___   __ _ _ __ ___  _ __ | | ___ 
      "||"       \___ \ / _` | '_ ` _ \| '_ \| |/ _ \ 
       || /\     ____) | (_| | | | | | | |_) | |  __/
    /\ ||//\)   |_____/ \__,_|_| |_| |_| .__/|_|\___|
   (/\||/                             |_|           
______\||/___________________________________________                     

ID: test-3166-0
Clear Text: 
 	text: Und prompt : die Besucherinnen und Besucher hatten Extrawünsche .
 	ner_label: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Tokenized: 
 	tokens: ['Und', 'prom', '##pt', ':', 'die', 'Besucher', '##innen', 'und', 'Besucher', 'hatten', 'Extra', '##wün', '##sche', '.']
 	offsets: [0, 4, 8, 11, 13, 17, 25, 31, 35, 44, 51, 56, 59, 64]
 	start_of_word: [True, True, False, True, True, True, False, True, True, True, True, False, False, True]
Fe

In [ ]:
model = trainer.train(model)